<a href="https://colab.research.google.com/github/OksanaSitalova/A-B-Testing-Analysis-From-Python-to-Interactive-Dashboard/blob/main/AB_TESTING_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import pandas as pd
import numpy as np

In [ ]:
# connecting to a csv file on Google Drive
from google.colab import drive
import os

drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset overview

In [ ]:
df = pd.read_csv('bq-results-20250610-052423-1749544006562.csv', parse_dates=['date'])
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


In [ ]:
# Basic info and preview

print("------- BASIC DATAFRAME INFO -------")
df.info()

print("\n------- SHAPE OF DATAFRAME -------")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

print("\n------- NULL VALUES CHECK -------")
print("Number of null (missing) values in each column:")
print(df.isnull().sum())

print("\n------- DUPLICATES CHECK -------")
print(f"Number of duplicate rows: {df.duplicated().sum()}")

------- BASIC DATAFRAME INFO -------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        800996 non-null  datetime64[ns]
 1   country     800996 non-null  object        
 2   device      800996 non-null  object        
 3   continent   800996 non-null  object        
 4   channel     800996 non-null  object        
 5   test        800996 non-null  int64         
 6   test_group  800996 non-null  int64         
 7   event_name  800996 non-null  object        
 8   value       800996 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 55.0+ MB

------- SHAPE OF DATAFRAME -------
Number of rows: 800996
Number of columns: 9

------- NULL VALUES CHECK -------
Number of null (missing) values in each column:
date          0
country       0
device        0
continent     0
channel       0
t

In [ ]:
df.describe()

,date,test,test_group,value
count,800996,800996.000000,800996.000000,800996.000000
mean,2020-12-07 00:33:34.715678720,2.831191,1.499608,9.475863
min,2020-11-01 00:00:00,1.000000,1.000000,1.000000
25%,2020-11-19 00:00:00,2.000000,1.000000,1.000000
50%,2020-12-05 00:00:00,3.000000,1.000000,2.000000
75%,2020-12-20 00:00:00,4.000000,2.000000,5.000000
max,2021-01-27 00:00:00,4.000000,2.000000,1575.000000
std,NaN,1.116940,0.500000,37.493267



Column **"date"**:

The date range covers the period from **November 1, 2020 to January 27, 2021**. This is approximately **3 months** of data, which is enough for trend analysis or short-term experiments.

Column **"test"**:

It has a value from 1 to 4, that is, it has **4 types of testing**.

Column **"test_group"**:

It has values ​​of 1 and 2, so we have **2 test groups** - 1 - control group, 2 - test group.

Column **"value"**:

The range of values ​​is from 1 to 1575.

The mean value of 9.48 is well below the maximum, and the 75th percentile (5.00) is well below the maximum. This indicates a right-skewed distribution. A large number of events have low values ​​(median 2), but there are also some events with very high values ​​(up to 1575).

The high standard deviation (37.49) confirms the significant spread of data in the value column.

In [ ]:
# Replace '(not set)' with 'Undefined' in the 'continent' column
df['continent'] = df['continent'].replace('(not set)', 'Undefined')

df['event_name'] = df['event_name'].replace('new account', 'new_account')
df['event_name'] = df['event_name'].replace('session with orders', 'session_with_orders')

print("--- Unique Values for Categorical Columns ---")

# Unique values for 'device'
print("\nUnique values of 'device' column:")
print(df['device'].unique())

# Unique values for 'continent'
print("\nUnique values of 'continent' column:")
print(df['continent'].unique())

# Unique values for 'channel'
print("\nUnique values of 'channel' column:")
print(df['channel'].unique())

# Unique values for 'event_name'
print("\nUnique values of 'event_name' column:")
print(df['event_name'].unique())

print("\n--- Count of Unique Values for 'country' ---")

# Count of unique values for 'country'
print("Count of unique values of 'country' column:")
print(df['country'].nunique())

--- Unique Values for Categorical Columns ---

Unique values of 'device' column:
['mobile' 'desktop' 'tablet']

Unique values of 'continent' column:
['Europe' 'Americas' 'Asia' 'Africa' 'Oceania' 'Undefined']

Unique values of 'channel' column:
['Organic Search' 'Social Search' 'Paid Search' 'Direct' 'Undefined']

Unique values of 'event_name' column:
['new_account' 'session_with_orders' 'session' 'user_engagement'
 'page_view' 'select_item' 'begin_checkout' 'first_visit' 'view_item'
 'view_promotion' 'scroll' 'session_start' 'view_search_results'
 'add_shipping_info' 'add_to_cart' 'select_promotion' 'add_payment_info'
 'click' 'view_item_list']

--- Count of Unique Values for 'country' ---
Count of unique values of 'country' column:
108


***Overview of Updated Categorical Data:***

**device:**

Unique values: [**'mobile', 'desktop', 'tablet'**]

The data includes three main device types, allowing for segmentation of user behavior analysis by device.

**continent:**

Unique values: [**'Europe', 'Americas', 'Asia', 'Africa', 'Oceania', 'Undefined'**]

The replacement of (not set) with Undefined was successfully performed. This ensures cleaner data for geographical analysis, separating known continents from those that were undefined.

**channel:**

Unique values: [**'Organic Search', 'Social Search', 'Paid Search', 'Direct', 'Undefined'**]

The replacement of (not set) with Undefined was also successful. This is a key dimension for analyzing the effectiveness of various marketing or traffic channels.

**event_name:**

Unique values: [**'new_account', 'session_with_orders', 'session', 'user_engagement', 'page_view', 'select_item', 'begin_checkout', 'first_visit', 'view_item', 'view_promotion', 'scroll', 'session_start', 'view_search_results', 'add_shipping_info', 'add_to_cart', 'select_promotion', 'add_payment_info', 'click', 'view_item_list'**]

All event names have been successfully converted to snake_case, which improves readability and data consistency. This list provides a comprehensive overview of actions users performed on the platform.

**country:**

Count of unique values: **108**
Insight: 108 unique countries indicate a broad geographical user base. This allows for detailed country-level analysis to identify regional specificities.


# Analysis of the 4 tests performed

In [ ]:
# Grouping by test with appropriate aggregations
table_4_tests = df.groupby('test').agg(
    start_date=('date', 'min'),
    end_date=('date', 'max'),
    duration_days=('date', lambda x: (x.max() - x.min()).days + 1),
    total_rows=('date', 'count'),
    unique_devices=('device', pd.Series.nunique),
    unique_countries=('country', pd.Series.nunique),
    unique_continents=('continent', pd.Series.nunique),
    unique_channels=('channel', pd.Series.nunique)
).reset_index()

table_4_tests

,test,start_date,end_date,duration_days,total_rows,unique_devices,unique_countries,unique_continents,unique_channels
0,1,2020-11-01,2020-11-26,26,140624,3,108,6,5
1,2,2020-11-01,2020-11-29,29,156791,3,108,6,5
2,3,2020-11-19,2020-12-20,32,200757,3,108,6,5
3,4,2020-12-05,2021-01-27,54,302824,3,108,6,5


***Key Observations:***

**Number of Tests** - The table presents data for four distinct tests (Test ID 1, 2, 3, 4).

**Test Duration** - Tests have varying durations: from 26 days (Test 1) to 54 days (Test 4).
Test 4 is the longest, which typically provides more data and allows for better detection of long-term effects or stabilization of results.

**Data Volume** - As the Test ID increases, so does the number of data rows:
Test 4 has the largest data volume, which is consistent with its longer duration. This is beneficial for statistical power.

*Audience Segment Coverage:*

**unique_devices** - All tests cover 3 unique devices. This means that all tests were conducted on the same device types (likely mobile, desktop, tablet, as we saw previously).

**unique_countries** - All tests were conducted in 108 unique countries. This confirms a broad geographical audience for each test.

**unique_continents** - All tests covered 6 unique continents. This indicates the global nature of each test.

**unique_channels** - All tests covered 5 unique channels. This suggests that traffic for each test came from the same primary sources.

**Timelines:** - Tests overlap in time:

Test 1 and 2 start on the same day (2020-11-01).
Test 3 starts later (2020-11-19) and extends into December.
Test 4 starts even later (2020-12-05) and runs until January 2021.
**This is important to consider: if tests are run concurrently or overlap, they might influence each other**, for example, if they affect the same user group.


***Overall Conclusion from this Overview:***

We have four different A/B tests that vary in duration and the volume of collected data. However, in terms of coverage of devices, countries, continents, and channels, each test covers the same audience segments. This simplifies the comparison of results between tests, as the basic characteristics of the audience participating in each test are consistent.

Test 4 is the most significant due to its duration and data volume, potentially providing the most reliable results. The temporal overlap between tests is a potential issue to keep in mind during analysis, but without a detailed understanding of the nature of the tests, it's difficult to assess its impact.


Checking the uniformity of distribution in test groups

In [ ]:
# Group by test number and group
distribution = (df.groupby(['test', 'test_group']).size().unstack(fill_value=0).rename(columns={1: 'Group 1', 2: 'Group 2'}))

# Calculation of interest
distribution_percent = (distribution.div(distribution.sum(axis=1), axis=0) * 100).round(2)

for test_id, row in distribution_percent.iterrows():
    print(f"Test {test_id}: Group 1 — {row['Group 1']}%, Group 2 — {row['Group 2']}%")


Test 1: Group 1 — 49.85%, Group 2 — 50.15%
Test 2: Group 1 — 50.58%, Group 2 — 49.42%
Test 3: Group 1 — 49.69%, Group 2 — 50.31%
Test 4: Group 1 — 50.08%, Group 2 — 49.92%



For each test, the groups are distributed almost evenly (approximately 50/50)

In [ ]:
# Function to calculate the percentage distribution across a selected variable
def compute_distribution_by(test_number, column):
    df_test = df[df['test'] == test_number]
    grouped = (
        df_test.groupby([column, 'test_group'])
        .size()
        .unstack(fill_value=0)
        .rename(columns={1: 'Group 1', 2: 'Group 2'})
    )
    percent = (grouped.div(grouped.sum(axis=1), axis=0) * 100).round(2)
    return percent

# Store the results for each test and variable
results = {}
for test_num in [1, 2, 3, 4]:
    results[f'Test {test_num} - by Continent'] = compute_distribution_by(test_num, 'continent')
    results[f'Test {test_num} - by Channel'] = compute_distribution_by(test_num, 'channel')
    results[f'Test {test_num} - by Device'] = compute_distribution_by(test_num, 'device')

# output for all tests and sections
for test_num in [1, 2, 3, 4]:
    for dim in ['continent', 'channel', 'device']:
        key = f"Test {test_num} - by {dim.capitalize()}"
        df_out = results[key]
        print(f"\n{key}")
        print(f"{dim.capitalize():<15} {'Group 1 (%)':>15} {'Group 2 (%)':>15}")
        print("-" * 45)
        for idx, row in df_out.iterrows():
            print(f"{str(idx):<15} {row['Group 1']:>15.2f} {row['Group 2']:>15.2f}")


Test 1 - by Continent
Continent           Group 1 (%)     Group 2 (%)
---------------------------------------------
Africa                    51.90           48.10
Americas                  49.88           50.12
Asia                      49.26           50.74
Europe                    50.18           49.82
Oceania                   50.38           49.62
Undefined                 49.63           50.37

Test 1 - by Channel
Channel             Group 1 (%)     Group 2 (%)
---------------------------------------------
Direct                    50.54           49.46
Organic Search            50.01           49.99
Paid Search               49.86           50.14
Social Search             49.63           50.37
Undefined                 47.98           52.02

Test 1 - by Device
Device              Group 1 (%)     Group 2 (%)
---------------------------------------------
desktop                   49.50           50.50
mobile                    50.31           49.69
tablet                    49.6

***Distribution in section***

**Continents:**

The distribution fluctuates somewhat, especially in Africa and Asia. For example:

In Africa for Test 2 — Group 1: 54.47%, Group 2: 45.53%

In Europe — the distribution is almost perfect across all tests (~50%)

**Channels:**

The largest shift — in the Undefined channel, where for example:

Test 1 — Group 1: 47.98%, Group 2: 52.02%

Other channels, such as Organic, Paid, Direct, — are close to equal distribution.

**Devices:**

Stable distribution across all device types (desktop, mobile, tablet)

For example, for mobile in Test 4 — Group 1: 50.12%, Group 2: 49.88%

In [ ]:
# list of metrics for the new table
new_metrics = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new_account",
    "first_visit",
    "select_promotion"
]

# Function to calculate a table with new metrics
def compute_ab_metrics_selected(df, metrics, session_event='session'):
    from statsmodels.stats.proportion import proportions_ztest

    results = []

    for test_number in sorted(df['test'].unique()):
        df_test = df[df['test'] == test_number]
        session_sums = df_test[df_test['event_name'] == session_event].groupby('test_group')['value'].sum()
        event_sums = df_test[df_test['event_name'].isin(metrics)].groupby(['test_group', 'event_name'])['value'].sum().unstack(fill_value=0)

        for metric in metrics:
            num_1 = event_sums.loc[1, metric] if metric in event_sums.columns and 1 in event_sums.index else 0
            num_2 = event_sums.loc[2, metric] if metric in event_sums.columns and 2 in event_sums.index else 0
            denom_1 = session_sums[1] if 1 in session_sums.index else 0
            denom_2 = session_sums[2] if 2 in session_sums.index else 0

            if denom_1 == 0 or denom_2 == 0:
                continue

            rate_1 = round(100 * num_1 / denom_1, 2)
            rate_2 = round(100 * num_2 / denom_2, 2)
            change = round((rate_2 / rate_1 - 1) * 100, 4) if rate_1 > 0 else 0

            z, p = proportions_ztest([num_1, num_2], [denom_1, denom_2])

            results.append({
                'test': test_number,
                'metric': metric,
                'numerator_1': int(num_1),
                'denominator_1': int(denom_1),
                'conversion_rate_group_1': rate_1,
                'numerator_2': int(num_2),
                'denominator_2': int(denom_2),
                'conversion_rate_group_2': rate_2,
                'metric_change_percentage': change,
                'z_stat': round(z, 4),
                'p_value': round(p, 6),
                'significant': p < 0.05
            })

    return pd.DataFrame(results)

selected_metrics_results = compute_ab_metrics_selected(df, new_metrics)
selected_metrics_results.head(24)

,test,metric,numerator_1,denominator_1,conversion_rate_group_1,numerator_2,denominator_2,conversion_rate_group_2,metric_change_percentage,z_stat,p_value,significant
0,1,add_payment_info,1988,45362,4.38,2229,45193,4.93,12.5571,-3.9249,0.000087,True
1,1,add_shipping_info,3034,45362,6.69,3221,45193,7.13,6.5770,-2.6036,0.009226,True
2,1,begin_checkout,3784,45362,8.34,4021,45193,8.90,6.7146,-2.9788,0.002894,True
3,1,new_account,3823,45362,8.43,3681,45193,8.15,-3.3215,1.5429,0.122859,False
4,1,first_visit,30596,45362,67.45,30512,45193,67.51,0.0890,-0.2131,0.831225,False
5,1,select_promotion,1275,45362,2.81,1323,45193,2.93,4.2705,-1.0520,0.292778,False
6,2,add_payment_info,2344,50637,4.63,2409,50244,4.79,3.4557,-1.2410,0.214608,False
7,2,add_shipping_info,3480,50637,6.87,3510,50244,6.99,1.7467,-0.7096,0.477979,False
8,2,begin_checkout,4262,50637,8.42,4313,50244,8.58,1.9002,-0.9529,0.340642,False
9,2,new_account,4165,50637,8.23,4184,50244,8.33,1.2151,-0.5888,0.556000,False


In [ ]:
# Saving a DataFrame to CSV
selected_metrics_results.to_csv("ab_test_selected_metrics.csv", index=False)


# Detailed Review by Test:

**Test 1:**

Significant Positive Changes: A statistically significant (p < 0.05) increase was observed for three metrics:
add_payment_info / session (+2.56%)
add_shipping_info / session (+6.58%)
begin_checkout / session (+6.71%)
Other Metrics: new_account, first_visit, select_promotion showed no statistically significant changes.
Conclusion: Test 1 was successful in improving key stages of the conversion funnel.

**Test 2:**

No Significant Changes: None of the tested metrics showed a statistically significant difference between the control and test groups (all p-values > 0.05).
Conclusion: Test 2 had no noticeable impact on any of the monitored metrics.

**Test 3:**

Significant Positive Changes: Only the begin_checkout / session metric showed a statistically significant increase (+2.51%, p = 0.012026).
Other Metrics: Showed no statistically significant changes.
Conclusion: Test 3 was partially successful, improving only the initiation of the checkout process.

**Test 4:**

Significant Positive Changes: A statistically significant increase was observed for:
begin_checkout / session (+1.99%, p = 0.045934)
new_account / session (+2.38%, p = 0.017527)
Other Metrics: Showed no statistically significant changes.
Conclusion: Test 4 contributed to improving the start of the checkout process and new account registrations.


**Key Conclusions and Recommendations:**

**Successful Tests: **

**Test 1, Test 3, and Test 4** showed a positive impact on various metrics.

begin_checkout / session Metric: This metric consistently improved in Tests 1, 3, and 4. This indicates that the changes implemented in these tests are likely effective in encouraging users to start the checkout process. It's worth analyzing in more detail what exactly was changed in these tests to scale this success.

**new_account / session Metric:** Only Test 4 showed a significant positive impact on new account creation.

**Test 2:** Despite being conducted, it did not yield statistically significant results. It's worth reviewing the hypothesis and implementation of this test.

**Non-Significant Metrics: **The first_visit / session and select_promotion / session metrics did not show significant changes in any of the tests. This could mean that the implemented changes do not affect these stages of the user journey, or that their sensitivity to changes was low.

The results of this analysis are visualized on a dashboard in Tableau Public.
https://public.tableau.com/app/profile/oksana.sitalova/viz/A_BTESTINGANALYSIS/Dashboard1?publish=yes